In [42]:
import random
import os
import glob
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision

from torchinfo import summary
from torch import nn
from torchvision import transforms
from torch.utils.data import (Dataset, DataLoader)

from PIL import Image
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from scikitplot.metrics import plot_roc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, precision_recall_fscore_support,
    accuracy_score, top_k_accuracy_score, f1_score, 
    matthews_corrcoef, confusion_matrix, ConfusionMatrixDisplay
)

In [44]:
class CFG:
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    NUM_DEVICES = torch.cuda.device_count()
    NUM_WORKERS = os.cpu_count()
    NUM_CLASSES = 45
    EPOCHS = 10
    BATCH_SIZE = (
        32 if torch.cuda.device_count() < 2 
        else (32 * torch.cuda.device_count())
    )
    LR = 0.001
    APPLY_SHUFFLE = True
    SEED = 768
    HEIGHT = 256
    WIDTH = 256
    CHANNELS = 3
    IMAGE_SIZE = (256, 256, 3)

In [45]:
# Define paths
PROJECT_ROOT = os.getcwd()
# DATASET_PATH = "./mammals"
DATASET_PATH = os.path.join(PROJECT_ROOT, "mammals")

TRAIN_PATH = DATASET_PATH


In [47]:
%%time
image_paths = glob.glob(f"{TRAIN_PATH}/**/*.jpg", recursive=True)

CPU times: user 23.1 ms, sys: 18 ms, total: 41.1 ms
Wall time: 40.3 ms


In [49]:
def extract_labels(image_paths):
    return [os.path.basename(os.path.dirname(path)) for path in image_paths]

dataset_df = pd.DataFrame({
    'image_path': image_paths,
    'label': extract_labels(image_paths)
})

print(dataset_df.head())  # View first 5 samples


                                          image_path       label
0  /Users/upekhansaja/Documents/Projects/Python/m...  rhinoceros
1  /Users/upekhansaja/Documents/Projects/Python/m...  rhinoceros
2  /Users/upekhansaja/Documents/Projects/Python/m...  rhinoceros
3  /Users/upekhansaja/Documents/Projects/Python/m...  rhinoceros
4  /Users/upekhansaja/Documents/Projects/Python/m...  rhinoceros


In [51]:
train_df, test_val_df = train_test_split(dataset_df, test_size=0.30, stratify=dataset_df['label'], random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.65, stratify=test_val_df['label'], random_state=42)

print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")


Train: 9625, Validation: 1444, Test: 2682


In [52]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [53]:
class MammalsDataset(Dataset):
    def __init__(self, df, transform=None):
        self.paths = df['image_path'].to_list()
        self.labels = df['label'].to_list()
        self.transform = transform
        self.classes = sorted(df['label'].unique())
        self.label_encoder = LabelEncoder().fit(self.classes)
        self.label_indices = self.label_encoder.transform(self.labels)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = Image.open(self.paths[idx]).convert('RGB')
        label = self.label_indices[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


In [54]:
train_dataset = MammalsDataset(train_df, transform=train_transforms)
val_dataset = MammalsDataset(val_df, transform=test_transforms)
test_dataset = MammalsDataset(test_df, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [55]:
def build_model():
    model = torchvision.models.efficientnet_v2_l(weights="DEFAULT")
    
    for param in model.features.parameters():
        param.requires_grad = False  # Freeze feature extractor
    
    model.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(1280, len(train_dataset.classes))
    )

    return model.to("cuda" if torch.cuda.is_available() else "cpu")


In [56]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(build_model().parameters(), lr=0.001)


In [ ]:
def train(model, train_loader, val_loader, epochs=10):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {correct / len(train_loader.dataset):.4f}")

train(build_model(), train_loader, val_loader, epochs=5)
















































































































































































































































































































100%|██████████| 301/301 [1:21:15<00:00, 16.20s/it]


Epoch 1, Loss: 1186.2393, Accuracy: 0.0209
















































































































































































































































































































100%|██████████| 301/301 [1:22:22<00:00, 16.42s/it]


Epoch 2, Loss: 1186.8788, Accuracy: 0.0224
